In [1]:
from teadata import DataEngine

# If needed:
from teadata.classes import haversine_miles, inspect_object


# Enable the Colab widget manager only when actually in Colab
def _enable_colab_widgets():
    try:
        from google.colab import output  # type: ignore

        output.enable_custom_widget_manager()
        print("[colab] custom widget manager enabled")
    except Exception:
        # Not in Colab (or not available) — nothing to do
        pass


_enable_colab_widgets()

import pandas as pd

import itables
from itables import show, init_notebook_mode, options
from itables.javascript import JavascriptFunction
from IPython.display import HTML, display
import pandas as pd

import sys, subprocess

pkgs = [
    "ipyleaflet==0.20.0",  # Python back-end package
    "jupyter-leaflet==0.20.0",  # Front-end lab extension (federated for JLab 4)
    "ipywidgets>=8.1",  # Widgets v8 (required by ipyleaflet 0.20)
    "jupyterlab-widgets>=3.0",
    "widgetsnbextension>=4.0",  # Classic NB support (harmless in JLab)
]

# subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", *pkgs])

import ipyleaflet, ipywidgets

print("ipyleaflet", ipyleaflet.__version__, "| ipywidgets", ipywidgets.__version__)

init_notebook_mode(all_interactive=True)

# Some sensible defaults (optional)
options.maxBytes = 0  # don't truncate
options.lengthMenu = [10, 25, 50, 100]
options.pageLength = 25

# Set option to display all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Enter the district name or district number of the school district you wish to analyze
DISTRICT_NAME = "Austin ISD"

# Instantiate DataEngine object, a layer over the District and Campus objects
repo = DataEngine.from_snapshot(search=True)

# Inspect how many objects are loaded
print(
    f"Loaded {len(repo.districts)} districts/charters and {len(repo.campuses)} campuses"
)

ImportError: cannot import name 'inspect_object' from 'teadata.classes' (/Users/adpena/PycharmProjects/teadata/teadata/classes.py)

In [ ]:
# Select a district using either the district name (case insensitive) or district number (any format works - integer and string with or without a leading apostrophe are both acceptable)
district = (repo >> ("district", DISTRICT_NAME)).first()
print("Example district:", district.name, district.district_number)

# Access canonical and enriched attributes (dot-syntax works if enrichment logic is wired correctly)
print(
    "\n2025 overall accountability rating:",
    district.rating,
    district.overall_rating_2025,
)

# Explore campuses inside the district
print(f"\n{district.name} has {len(district.campuses)} total campuses.")
print(f"\n{district.name} campus ratings:", district.campuses.value_counts("rating"))
print(f"\n{district.name} school types:", district.campuses.value_counts("school_type"))

In [ ]:
# Select all of a district's campuses, then filter and sort and take the top 5 by enrollment, then find the nearest charter school to each campus with the same "school type" ("Elementary School", "Junior High School", "Middle School", "Elementary/Secondary", "High School")

# TODO: Is a better comparison to find overlapping grades served?

top5 = (
    repo
    >> ("campuses_in", district)
    >> ("where", lambda c: (c.enrollment or 0) > 0)
    >> ("sort", lambda c: c.enrollment or 0, True)
    >> ("take", 5)
    >> ("nearest_charter_same_type",)  # yields list of (campus, match, miles)
)

In [ ]:
# Each data type includes .to_dict() (or .to_dicts() if a list) and .to_df() helper functions for exporting or working with query results
df = top5.to_df()  # auto-flattened & prefixed

# Define the columns we are interested in and the order we want
cols = [
    "campus_name",
    "campus_enrollment",
    "campus_rating",
    "campus_school_type",
    "match_name",
    "match_enrollment",
    "match_rating",
    "match_school_type",
    "distance_miles",
]

# Apply filter/sort to dataframe
df = df[cols]

# Display dataframe
df

In [ ]:
# Get charter campuses physically inside that district’s boundary - either syntax works
charters_within = repo.charter_campuses_within(district)
charters_within = repo >> ("charters_within", district)

print(
    f"{district.name}: {len(charters_within)} charter campuses inside district boundary"
)

# Build the charter-destination query
charter_rows = (
    repo
    >> ("district", "Austin ISD")
    >> ("campuses_in",)  # start from Austin ISD-operated campuses
    >> ("transfers_out", True)  # expand to outgoing transfer edges, charter-only
    >> ("where_to", lambda to_: to_ is not None)
    >> ("distinct", lambda row: getattr(row[1], "id", None))
)

# Materialize the unique charter campuses receiving Austin ISD transfers
charter_destinations = charter_rows >> ("map", lambda row: row[1])

# Aggregations
print(
    f"{district.name}: {len(charter_destinations)} unique charter campuses receiving student transfers"
)

privates_within = repo >> ("privates_within", district)

print(
    f"\n{district.name}: {len(privates_within)} private school campuses inside district boundary"
)

privates_in = repo >> ("private_campuses_in", district)

print(
    f"{district.name}: {len(privates_in)} total private school campuses associated w/ district per TEPSAC"
)

total_charter_enrollment_within = sum(
    (campus.enrollment or 0) for campus in charters_within
)
print(
    f"\nCombined charter enrollment (INSIDE {district.name} BOUNDARY): {total_charter_enrollment_within:,}"
)

total_charter_enrollment_all = sum(
    (campus.enrollment or 0) for campus in charter_destinations
)
print(
    f"Combined charter enrollment (INSIDE & OUTSIDE {district.name} BOUNDARY): {total_charter_enrollment_all:,}"
)

total_private_enrollment = sum((campus.enrollment or 0) for campus in privates_in)
print(
    f"\nCombined private enrollment (INSIDE & OUTSIDE {district.name} BOUNDARY): {total_private_enrollment:,}"
)

print("\n----------------------------\n")

for campus in charter_destinations:
    print(campus.campus_number, campus.name)

In [ ]:
# Similar to before, export the result of our query to a dataframe for display after filtering and sorting the columns

df = charters_within.to_df()

# print(list(df.columns))

cols = [
    "name",
    "campus_number",
    "district_number",
    "charter_type",
    "aea",
    "enrollment",
    "overall_rating_2025",
    "grade_range",
    "school_type",
    "school_status_date",
    "update_date",
]

df = df[cols]
df = df.sort_values(by="name")

df

In [ ]:
# Using the District object, select the campuses in the school district with unmasked enrollment,
# sort in descending order by enrollment, take the five largest campuses, then expand to student
# transfers out BUT only to charter campuses (filter applied inside the operator).

rows = (
    repo
    >> ("campuses_in", district)
    >> ("where", lambda x: (x.enrollment or 0) > 0)
    >> ("sort", lambda x: x.enrollment or 0, True)
    # >> ("take", 5)
    >> ("transfers_out", True)  # Option 1: charter_only=True baked in
)

# 1) Build DF as you do now
df = rows.to_df()

# print(list(df.columns))

cols = [
    "campus_campus_number",
    "campus_name",
    "campus_enrollment",
    "campus_students_w_disabilities_af_per_student",
    "campus_overall_rating_2025",
    "campus_aea",
    "campus_grade_range",
    "campus_school_type",
    "campus_school_status_date",
    "campus_update_date",
    "to_campus_number",
    "to_name",
    "to_enrollment",
    "to_students_w_disabilities_af_per_student",
    "to_overall_rating_2025",
    "to_aea",
    "to_grade_range",
    "to_school_type",
    "to_charter_type",
    "to_is_charter",
    "to_school_status_date",
    "to_update_date",
    "count",
    "masked",
]
df = df[cols].copy()

itables.options.warn_on_undocumented_option = False

# Ensure numeric
lhs = "campus_students_w_disabilities_af_per_student"
rhs = "to_students_w_disabilities_af_per_student"
df[lhs] = pd.to_numeric(df[lhs], errors="coerce")
df[rhs] = pd.to_numeric(df[rhs], errors="coerce")

# 1) Find column indexes (DataTables passes row data as an array)
idx_a = df.columns.get_loc(lhs)
idx_b = df.columns.get_loc(rhs)

# 2) Shade rows using rowCallback (fires on draw/order/paging too)
row_cb = JavascriptFunction(
    f"""
function(row, data, dataIndex) {{
  const a = Number(data[{idx_a}]);
  const b = Number(data[{idx_b}]);
  if (!Number.isNaN(a) && !Number.isNaN(b)) {{
    if (a > b) row.classList.add('row-green');
    else if (a < b) row.classList.add('row-red');
  }}
}}
"""
)

# 3) CSS for row coloring (keep it once per notebook)
display(
    HTML(
        """
<style>
table.dataTable tbody tr.row-green { background-color: #e6ffe6 !important; }
table.dataTable tbody tr.row-red   { background-color: #ffe6e6 !important; }

/* Optional: keep header above content when fixedHeader is active */
table.dataTable thead th { background: white; }
</style>
"""
    )
)

# 4) Show with sticky header + freeze first two columns
show(
    df,
    scrollX=True,
    scrollY="600px",
    fixedHeader=True,  # sticky header
    fixedColumns={"leftColumns": 2},  # freeze first 2 columns (portable)
    rowCallback=row_cb,  # ✅ conditional row shading
    columnDefs=[{"targets": "_all", "className": "dt-left"}],
)

In [ ]:
# --- Inputs expected ---
# repo: DataEngine
# district: a District object
# df: DataFrame from your pipeline (rows.to_df()), filtered to the columns you care about and with "count" numeric

from ipyleaflet import Map, Marker, CircleMarker, Polyline, LayerGroup, basemaps, Popup
from ipywidgets import HTML
import pandas as pd
import math


# Helper: robust campus-number key lookup against repo (handles leading apostrophe or not)
def _canon_cnum(c):
    if c is None:
        return None
    s = str(c)
    return s if s.startswith("'") else "'" + s  # your objects use leading apostrophe


# ✅ Build a lookup from campus_number -> Campus object (with and without leading apostrophe)
campus_index = {}
# Prefer internal dict if present; else use the public view's .values()
if hasattr(repo, "_campuses") and isinstance(repo._campuses, dict):
    campus_iter = repo._campuses.values()
else:
    # repo.campuses is a ReadOnlyEntityView; use .values() to iterate Campus objects
    campus_iter = getattr(repo, "campuses", []).values()  # type: ignore[attr-defined]

for c in campus_iter:
    cnum = getattr(c, "campus_number", None)
    if not cnum:
        continue
    s = str(cnum)  # e.g. "'227901013"
    s2 = s.lstrip("'")  # e.g. "227901013"
    campus_index[s] = c  # with apostrophe
    campus_index[s2] = c  # without apostrophe


def get_campus(cnum):
    if cnum is None:
        return None
    s = str(cnum)
    # try as-is, stripped, and canonical with apostrophe re-added
    return (
        campus_index.get(s)
        or campus_index.get(s.lstrip("'"))
        or campus_index.get("'" + s.lstrip("'"))
    )


def get_latlon(cnum):
    c = get_campus(cnum)
    if not c:
        return None
    coords = getattr(c, "coords", None)
    if coords is None:
        return None

    # Accept either tuple/list or shapely-like point
    try:
        if hasattr(coords, "x") and hasattr(coords, "y"):
            # Shapely Point: x=lon, y=lat
            lon = float(coords.x)
            lat = float(coords.y)
        else:
            a, b = coords  # repo may store either (lon, lat) or (lat, lon)
            a = float(a)
            b = float(b)
            # First assume (lon, lat)
            lon = a
            lat = b
    except Exception:
        return None

    def _is_valid_latlon(lat, lon):
        return (-90.0 <= lat <= 90.0) and (-180.0 <= lon <= 180.0)

    def _looks_like_texas(lat, lon):
        return (25.0 <= lat <= 37.0) and (-107.0 <= lon <= -93.0)

    # Candidate 1: as parsed above (lon, lat) -> (lat, lon)
    cand1 = (lat, lon)
    # Candidate 2: flipped interpretation
    cand2 = (lon, lat)

    # Prefer the candidate that looks like Texas; else any valid lat/lon; else None
    if _looks_like_texas(*cand1):
        return cand1
    if _looks_like_texas(*cand2):
        return cand2
    if _is_valid_latlon(*cand1):
        return cand1
    if _is_valid_latlon(*cand2):
        return cand2
    return None


# Ensure numeric transfer count (for line thickness)
if "count" in df.columns:
    try:
        df["count"] = pd.to_numeric(df["count"], errors="coerce").fillna(0).astype(int)
    except Exception:
        df["count"] = "MASKED"

# Compute unique origin/destination sets and aggregate lines (origin -> dest)
origins = (
    df[["campus_campus_number", "campus_name"]]
    .drop_duplicates()
    .rename(columns={"campus_campus_number": "cnum", "campus_name": "name"})
)

dests = (
    df[["to_campus_number", "to_name"]]
    .drop_duplicates()
    .rename(columns={"to_campus_number": "cnum", "to_name": "name"})
)

# Build (origin, dest) edges with counts aggregated
edges = (
    df.groupby(["campus_campus_number", "to_campus_number"], dropna=False)["count"]
    .sum()
    .reset_index()
    .rename(
        columns={"campus_campus_number": "src", "to_campus_number": "dst", "count": "n"}
    )
)

# Precompute center (district centroid if available; else mean of lat/lon)
center = None
try:
    if getattr(district, "boundary", None) and getattr(
        district.boundary, "centroid", None
    ):
        ctr = district.boundary.centroid  # Shapely geometry
        center = (ctr.y, ctr.x)
except Exception:
    pass

if center is None:
    coords_list = [get_latlon(c) for c in origins["cnum"].tolist()]
    coords_list = [xy for xy in coords_list if xy]
    if coords_list:
        lat_mean = sum(lat for lat, lon in coords_list) / len(coords_list)
        lon_mean = sum(lon for lat, lon in coords_list) / len(coords_list)
        center = (lat_mean, lon_mean)
    else:
        center = (31.0, -99.0)  # Texas-ish fallback

# --- quick sanity on your inputs / lookups ---
print("[diag] rows in df:", len(df))
print("[diag] unique origins in df:", df["campus_campus_number"].nunique())
print("[diag] unique dests in df:", df["to_campus_number"].nunique())

# How many resolve to lat/lon?
orig_ok = sum(1 for c in origins["cnum"] if get_latlon(c))
dest_ok = sum(1 for c in dests["cnum"] if get_latlon(c))
print(f"[diag] origins with coords: {orig_ok}/{len(origins)}")
print(f"[diag] dests with coords:   {dest_ok}/{len(dests)}")

# Peek at a couple of resolved lat/lons for sanity (should be ~25..37, -107..-93 for TX)
try:
    sample_latlons = [get_latlon(c) for c in list(origins["cnum"])[:3] if get_latlon(c)]
    if sample_latlons:
        print("[diag] sample lat/lon:", sample_latlons[:3])
except Exception:
    pass

# If nothing resolves, show a few examples to inspect campus_number formatting
if orig_ok == 0 or dest_ok == 0:
    print("[diag] sample origins:", origins["cnum"].head(5).tolist())
    print("[diag] sample dests:  ", dests["cnum"].head(5).tolist())

# Create map
# Start on a safe Texas-wide view; precise bounds will be applied right after
m = Map(center=(31.0, -99.0), zoom=6, basemap=basemaps.CartoDB.Positron)
m.layout.height = "650px"

# Build destination -> list of (origin_name, origin_cnum, count) for popup aggregation
dest_sources = {}
for _, row in edges.iterrows():
    dst = row["dst"]
    src = row["src"]
    n = int(row["n"] or 0)
    src_obj = get_campus(src)
    if dst not in dest_sources:
        dest_sources[dst] = []
    dest_sources[dst].append(
        (src_obj.name if src_obj else str(src), _canon_cnum(src), n)
    )

# Create destination markers (charter campuses) with popups
dest_markers = {}
for _, r in dests.iterrows():
    cnum = r["cnum"]
    name = r["name"]
    latlon = get_latlon(cnum)
    if not latlon:
        continue

    # Popup: list all incoming sources with counts
    rows_html = ""
    for src_name, src_cnum, n in sorted(
        dest_sources.get(cnum, []), key=lambda x: -x[2]
    ):
        rows_html += (
            f"<tr><td>{src_name}</td><td style='text-align:right'>{n:,}</td></tr>"
        )
    if not rows_html:
        rows_html = (
            "<tr><td colspan='2'><em>No transfers (in filtered set)</em></td></tr>"
        )

    pop = Popup(
        child=HTML(
            value=f"""
        <div style="font-weight:600;margin-bottom:6px;">{name}</div>
        <div style="font-size:12px;margin-bottom:6px;">Campus #: { _canon_cnum(cnum) }</div>
        <div style="max-height:240px; overflow:auto; border-top:1px solid #ddd; padding-top:4px;">
          <table style="font-size:12px; width:100%;">
            <thead><tr><th align="left">From campus</th><th align="right">Count</th></tr></thead>
            <tbody>{rows_html}</tbody>
          </table>
        </div>
        """
        ),
        max_width=360,
        close_button=True,
    )

    mk = CircleMarker(
        location=latlon,
        radius=5,
        color="#b30000",
        fill_color="#b30000",
        fill_opacity=0.9,
        opacity=0.9,
        title=name,  # simple hover text
    )
    mk.popup = pop

    m.add_layer(mk)
    dest_markers[_canon_cnum(cnum)] = mk

# For each origin, create a marker and a LayerGroup of its outgoing lines (toggle on click)
origin_markers = {}
origin_line_groups = {}

# Normalize line thickness
max_n = int(edges["n"].max()) if len(edges) and edges["n"].max() > 0 else 1


def width_for(n):
    # scale 1..max_n to sensible stroke widths
    return max(1.5, 6.0 * (float(n) / float(max_n)))


for _, r in origins.iterrows():
    cnum = r["cnum"]
    name = r["name"]
    latlon = get_latlon(cnum)
    if not latlon:
        continue

    # Origin marker (district campus)
    mk = CircleMarker(
        location=latlon,
        radius=6,
        color="#0057b7",
        fill_color="#0057b7",
        fill_opacity=0.9,
        opacity=0.9,
        title=name,  # simple hover text
    )
    mk.popup = Popup(
        child=HTML(
            value=f"<b>{name}</b><br>Campus #: {_canon_cnum(cnum)}<br><i>Click to toggle transfers</i>"
        ),
        max_width=250,
    )
    m.add_layer(mk)
    origin_markers[_canon_cnum(cnum)] = mk

    # All outgoing lines for this origin (initially hidden)
    segs = []
    sub = edges[edges["src"].astype(str) == str(cnum)]
    for _, e in sub.iterrows():
        dst = e["dst"]
        n = int(e["n"] or 0)
        latlon_dst = get_latlon(dst)
        if not latlon_dst:
            continue
        segs.append(
            Polyline(
                locations=[latlon, latlon_dst],
                color="#666",
                weight=int(width_for(n)),
                opacity=0.75,
            )
        )

    group = LayerGroup(layers=tuple(segs))  # not added yet
    origin_line_groups[_canon_cnum(cnum)] = group

    # Click handler toggles the line group on/off
    def _make_click_handler(origin_key):
        def _on_click(**kwargs):
            grp = origin_line_groups.get(origin_key)
            if not grp:
                return
            if grp in m.layers:
                m.remove_layer(grp)
            else:
                m.add_layer(grp)

        return _on_click

    mk.on_click(_make_click_handler(_canon_cnum(cnum)))

# --- Fit view using district boundary first, else valid marker bounds, else TX fallback ---
import asyncio


def _valid_tx_latlon(ll):
    if not ll or len(ll) != 2:
        return False
    lat, lon = ll
    try:
        lat = float(lat)
        lon = float(lon)
    except Exception:
        return False
    return (25.0 <= lat <= 37.0) and (-107.0 <= lon <= -93.0)


def _fit_bounds_latlon(map_obj, latlons, pad=0.02):
    lats = [float(lat) for (lat, lon) in latlons]
    lons = [float(lon) for (lat, lon) in latlons]
    sw = (min(lats) - pad, min(lons) - pad)  # (south, west)
    ne = (max(lats) + pad, max(lons) + pad)  # (north, east)
    map_obj.fit_bounds([sw, ne])


# Use actual rendered marker locations for reliability
def _collect_all_latlons():
    latlons = []
    # Use the actual rendered marker locations for reliability
    for mk in origin_markers.values():
        if mk and getattr(mk, "location", None) and _valid_tx_latlon(mk.location):
            latlons.append(tuple(mk.location))
    for mk in dest_markers.values():
        if mk and getattr(mk, "location", None) and _valid_tx_latlon(mk.location):
            latlons.append(tuple(mk.location))
    return latlons


# 1) Prefer district boundary if present and sane
preferred_bounds = None
try:
    poly = getattr(district, "boundary", None)
    if poly is not None and hasattr(poly, "bounds"):
        minx, miny, maxx, maxy = poly.bounds  # lon/lat
        sw = (float(miny), float(minx))
        ne = (float(maxy), float(maxx))
        if _valid_tx_latlon(sw) and _valid_tx_latlon(ne):
            preferred_bounds = [sw, ne]
except Exception:
    pass

# 2) Else use all origin+dest points
if preferred_bounds is None:
    all_latlons = _collect_all_latlons()
    if all_latlons:
        lats = [lat for (lat, lon) in all_latlons]
        lons = [lon for (lat, lon) in all_latlons]
        sw = (min(lats) - 0.02, min(lons) - 0.02)
        ne = (max(lats) + 0.02, max(lons) + 0.02)
        preferred_bounds = [sw, ne]

# 3) Fallback: Texas frame
if preferred_bounds is None:
    preferred_bounds = [(25.0, -107.0), (37.0, -93.0)]

# Apply immediately
m.fit_bounds(preferred_bounds)
# Best effort: force Leaflet to recalc container size before async nudges
try:
    m.invalidate_size()
except Exception:
    pass


# And nudge a couple more times after render to avoid "stuck" global view in some JLab setups
async def _nudge_bounds(bounds, attempts=3, delay=0.08):
    for _ in range(attempts):
        await asyncio.sleep(delay)
        m.fit_bounds(bounds)


asyncio.create_task(_nudge_bounds(preferred_bounds))

print("[view] bounds applied:", preferred_bounds)

In [ ]:
m

In [ ]:
from pprint import pprint

rows = (
    repo
    >> ("campuses_in", district)
    >> ("where", lambda x: (x.enrollment or 0) > 0)
    >> ("where", lambda x: x.facing_closure is True)
    >> ("sort", lambda x: x.name or "", False)
    # >> ("take", 5)
    # >> ("transfers_out", True)  # Option 1: charter_only=True baked in
    >> (
        "nearest_charter_transfer_destination",
    )  # yields list of (campus, match, miles)
)

for closure_campus in rows:
    match = closure_campus[1]
    distance = closure_campus[2]
    closure_campus = closure_campus[0]

    print(
        closure_campus.campus_number,
        closure_campus.name,
        closure_campus.enrollment,
        closure_campus.rating,
        closure_campus.aea,
        closure_campus.grade_range,
    )

df = rows.to_df()

# pprint(list(df.columns))

cols = {
    "campus_number": "Campus Number",
    "name": "Campus Name",
    "enrollment": "Enrollment as of Oct 2024",
    "overall_rating_2025": "2025 Overall Rating",
    "aea": "AEA",
    "grade_range": "Grade Range",
    "percent_enrollment_change": "2014-15 to 2024-2025 Percent Enrollment Change",
    "num_charter_transfer_destinations": "Number of Charter Campuses Receiving Student Transfers - TOTAL",
    "num_charter_transfer_destinations_masked": "Number of Charter Campuses Receiving Student Transfers - MASKED",
    "total_unmasked_charter_transfers_out": "Total Unmasked Charter Transfers Out",
    "closure_date": "Proposed Closure Date",
    "campus_2024_student_membership_2023_mobility_all_students_percent": "2024 Student Membership: 2023 Mobility All Students Percent",
    "campus_2024_student_membership_2022_23_attrition_all_students_percent": "2024 Student Membership: 2022-23 Attrition All Students Percent",
    "campus_2024_student_enrollment_african_american_percent": "2024 Student Enrollment: African American Percent",
    "campus_2024_student_enrollment_hispanic_percent": "2024 Student Enrollment: Hispanic Percent",
    "campus_2024_student_enrollment_econ_disadv_percent": "2024 Student Enrollment: Econ Disadvantaged Percent",
    "campus_2024_student_enrollment_section_504_percent": "2024 Student Enrollment: Section 504 Percent",
    "campus_2024_student_enrollment_el_percent": "2024 Student Enrollment: EL Percent",
    "campus_2024_student_enrollment_dyslexia_percent": "2024 Student Enrollment: Dyslexia Percent",
    "campus_2024_student_enrollment_foster_care_percent": "2024 Student Enrollment: Foster Care Percent",
    "campus_2024_student_enrollment_homeless_percent": "2024 Student Enrollment: Homeless Percent",
    "campus_2024_student_enrollment_immigrant_percent": "2024 Student Enrollment: Immigrant Percent",
    "campus_2024_student_enrollment_migrant_percent": "2024 Student Enrollment: Migrant Percent",
    "campus_2024_student_enrollment_title_i_percent": "2024 Student Enrollment: Title I Percent",
    "campus_2024_student_enrollment_at_risk_percent": "2024 Student Enrollment: At Risk Percent",
    "campus_2024_student_enrollment_bilingual_esl_percent": "2024 Student Enrollment: Bilingual ESL Percent",
    "campus_2024_student_enrollment_special_ed_percent": "2024 Student Enrollment: Special Ed Percent",
    "campus_2024_staff_teacher_beginning_full_time_equiv_percent": "2024 Staff: Teacher Beginning FTE Percent",
    "campus_2024_staff_teacher_1_5_years_full_time_equiv_percent": "2024 Staff: Teacher 1-5 Years FTE Percent",
    "campus_2024_staff_teacher_no_degree_full_time_equiv_percent": "2024 Staff: Teacher No Degree FTE Percent",
    "campus_2024_staff_teacher_ba_degree_full_time_equiv_percent": "2024 Staff: Teacher BA Degree FTE Percent",
    "campus_2024_staff_teacher_student_ratio": "2024 Staff: Teacher to Student Ratio",
}

# 1) Build ordered lists, keeping only columns that actually exist in df
campus_order = [f"campus_{c}" for c in cols.keys() if f"campus_{c}" in df.columns]
match_order = [f"match_{c}" for c in cols.keys() if f"match_{c}" in df.columns]

# 2) Handle distance column(s) gracefully (pick first present)
distance_candidates = ["distance_miles", "distance", "miles"]
distance_col = next((c for c in distance_candidates if c in df.columns), None)
distance_order = [distance_col] if distance_col else []

# 3) Final ordered columns: campus_* then match_* then distance
ordered_cols = campus_order + match_order + distance_order

# 4) Filter & copy
_df = df[ordered_cols].copy()

# 5) Build rename map:
#    - campus_<key> -> cols[<key>]
#    - match_<key>  -> "NEAREST CHARTER " + cols[<key>]
#    - distance     -> "Distance in Miles"
rename_map = {}
for key, label in cols.items():
    ccol = f"campus_{key}"
    mcol = f"match_{key}"
    if ccol in _df.columns:
        rename_map[ccol] = label
    if mcol in _df.columns:
        rename_map[mcol] = f"NEAREST CHARTER {label}"

if distance_col and distance_col in _df.columns:
    rename_map[distance_col] = "Distance in Miles"

# 6) Apply renames
df = _df.rename(columns=rename_map)

print(len(df.columns))

# 7) Export
"""df.to_excel(
    "Austin ISD Consolidation Plan_Exploratory Analysis_10.2025.xlsx", index=False
)"""